In [1]:
import pandas as pd

In [2]:
train= pd.read_csv("train.csv")

In [15]:
positive_size = train[train.target==1].shape[0]

In [17]:
positive_percentage = positive_size/float(train.shape[0])

In [25]:
print "%.2f" % (positive_percentage*100), "% of the data is labelled as positive."

3.64 % of the data is labelled as positive.


## Missing values

In [11]:
missing_values = train.apply(lambda x: len(x[x==-1])/float(len(x)), axis=0)

In [14]:
missing_values.sort_values(ascending=False)

ps_car_03_cat     0.690898
ps_car_05_cat     0.447825
ps_reg_03         0.181065
ps_car_14         0.071605
ps_car_07_cat     0.019302
ps_ind_05_cat     0.009760
ps_car_09_cat     0.000956
ps_ind_02_cat     0.000363
ps_car_01_cat     0.000180
ps_ind_04_cat     0.000139
ps_car_02_cat     0.000008
ps_car_11         0.000008
ps_car_12         0.000002
ps_ind_03         0.000000
ps_ind_18_bin     0.000000
target            0.000000
ps_ind_01         0.000000
ps_reg_02         0.000000
ps_reg_01         0.000000
ps_ind_16_bin     0.000000
ps_ind_17_bin     0.000000
ps_ind_15         0.000000
ps_car_04_cat     0.000000
ps_ind_13_bin     0.000000
ps_ind_12_bin     0.000000
ps_ind_11_bin     0.000000
ps_ind_10_bin     0.000000
ps_ind_09_bin     0.000000
ps_ind_08_bin     0.000000
ps_ind_07_bin     0.000000
ps_ind_06_bin     0.000000
ps_ind_14         0.000000
ps_calc_20_bin    0.000000
ps_car_06_cat     0.000000
ps_calc_19_bin    0.000000
ps_calc_18_bin    0.000000
ps_calc_17_bin    0.000000
p